In [17]:
import os
import sys

app_path = os.getcwd().rsplit(os.sep, 1)[0]

if app_path not in sys.path:
    sys.path.insert(0, app_path)

In [18]:
import pandas as pd

from utils.paths import raw_data_path
from utils.helpers import *

In [19]:
path = raw_data_path('train.csv')
df   = pd.read_csv(path)

df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,5728806,Large private room in Nolita,4271676,Nat,Manhattan,Nolita,40.72217,-73.99481,Private room,120,7,3,2015-09-01,0.06,3,0
1,2243769,Super Bowl New York City Apartment,11460768,Brian,Manhattan,Upper West Side,40.80020,-73.96045,Entire home/apt,1500,1,0,NaN,NaN,1,0
2,35515415,COZY APT CLOSE TO LGA & JFK AND TRANSPORTATION,267193767,Jenny,Queens,East Elmhurst,40.75558,-73.89316,Entire home/apt,200,2,4,2019-07-05,4.00,1,365
3,36202006,Stream-Pressed Paradise (Laundry Room Setup),43392243,Rita,Staten Island,Concord,40.60110,-74.07830,Shared room,30,2,0,NaN,NaN,4,82
4,3780951,Charming Harlem apartment,16065171,Gina,Manhattan,Harlem,40.81022,-73.94266,Shared room,115,1,16,2019-05-26,0.27,1,365


## Filling NaN values

In [20]:
missing_values_percent(df)

id                                 0.000000
name                               0.035061
host_id                            0.000000
host_name                          0.037983
neighbourhood_group                0.000000
neighbourhood                      0.000000
latitude                           0.000000
longitude                          0.000000
room_type                          0.000000
price                              0.000000
minimum_nights                     0.000000
number_of_reviews                  0.000000
last_review                       20.399696
reviews_per_month                 20.399696
calculated_host_listings_count     0.000000
availability_365                   0.000000
dtype: float64

In [21]:
# Drop name column
df = df.drop("name", 1)

# Filling unknown host-name with "AAA"
df.host_name = df.host_name.fillna("AAA")

# Replacing missing reviews-per-month with mean reviews per month 
mean_rpm = df.reviews_per_month.mean()
df.reviews_per_month = df.reviews_per_month.fillna(mean_rpm)

# Missing last-review dates filled with arbitrary date ('2001-01-01')
df.last_review = df.last_review.fillna('2001-01-01')

In [22]:
missing_values_percent(df)

id                                0.0
host_id                           0.0
host_name                         0.0
neighbourhood_group               0.0
neighbourhood                     0.0
latitude                          0.0
longitude                         0.0
room_type                         0.0
price                             0.0
minimum_nights                    0.0
number_of_reviews                 0.0
last_review                       0.0
reviews_per_month                 0.0
calculated_host_listings_count    0.0
availability_365                  0.0
dtype: float64

## Label Encoding DF

In [23]:
get_categorical_df(df).head()

,host_name,neighbourhood_group,neighbourhood,room_type,last_review
0,Nat,Manhattan,Nolita,Private room,2015-09-01
1,Brian,Manhattan,Upper West Side,Entire home/apt,2001-01-01
2,Jenny,Queens,East Elmhurst,Entire home/apt,2019-07-05
3,Rita,Staten Island,Concord,Shared room,2001-01-01
4,Gina,Manhattan,Harlem,Shared room,2019-05-26


In [24]:
cols =['host_name', 'neighbourhood_group', 'neighbourhood', 'room_type']
get_categorical_df(df).columns

Index(['host_name', 'neighbourhood_group', 'neighbourhood', 'room_type',
       'last_review'],
      dtype='object')

In [25]:
df = label_encode(df, cols)

In [26]:
df.head()

,id,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,5728806,4271676,6215,2,143,40.72217,-73.99481,1,120,7,3,2015-09-01,0.060000,3,0
1,2243769,11460768,1221,2,200,40.80020,-73.96045,0,1500,1,0,2001-01-01,1.377074,1,0
2,35515415,267193767,3940,3,59,40.75558,-73.89316,0,200,2,4,2019-07-05,4.000000,1,365
3,36202006,43392243,7185,4,46,40.60110,-74.07830,2,30,2,0,2001-01-01,1.377074,4,82
4,3780951,16065171,3052,2,93,40.81022,-73.94266,2,115,1,16,2019-05-26,0.270000,1,365


In [27]:
X = df.drop(['price'], 1)
y = df.price

result = run_regression_models(X, y, None)

ValueError: could not convert string to float: '2015-10-21'

## END